## Create annotations

In [ ]:
import os
import json
import omero.clients
from omero.gateway import BlitzGateway

In [ ]:
thisdir = os.path.abspath('')
interim_data = os.path.join(thisdir, '..\\data\\interim')
download_data = os.path.join(thisdir, 'download')

#### Create a connection

In [ ]:
server = "  "
username = " "
password = " "

In [ ]:
conn = BlitzGateway(username, password, host=server, port=4064)
conn.connect()

In [ ]:
user = conn.getUser()
print("Current user:")
print("   ID:", user.getId())
print("   Username:", user.getName())
print("   Full Name:", user.getFullName())

In [ ]:
my_exp_id = conn.getUser().getId()
default_group_id = conn.getEventContext().groupId

In [ ]:
def print_obj(obj, indent=0):
    """
    Helper method to display info about OMERO objects.
    Not all objects will have a "name" or owner field.
    """
    print("""%s%s:%s  Name:"%s" (owner=%s)""" % (
        " " * indent,
        obj.OMERO_CLASS,
        obj.getId(),
        obj.getName(),
        obj.getOwnerOmeName()))

In [ ]:
for project in conn.getObjects("Project", opts={'owner': my_exp_id,
                                            'group': default_group_id,
                                            'order_by': 'lower(obj.name)',
                                            'limit': 100, 'offset': 0}):
    print_obj(project)
    # We can get Datasets with listChildren, since we have the Project already.
    # Or conn.getObjects("Dataset", opts={'project', id}) if we have Project ID
    for dataset in project.listChildren():
        print_obj(dataset, 2)
        for image in dataset.listChildren():
            print_obj(image, 4)

- Create a new Dataset

In [ ]:
new_dataset = omero.gateway.DatasetWrapper(conn, omero.model.DatasetI())
new_dataset.setName('Test_DataSet')
new_dataset.save()
print("New dataset, Id:", new_dataset.id)
# Can get the underlying omero.model.DatasetI with:
dataset_obj = new_dataset._obj

- set object ID

In [ ]:
project_Id = 1
image_Id = 29

- Link to project

In [ ]:
link = omero.model.ProjectDatasetLinkI()
# We can use a 'loaded' object, but we might get an Exception
# link.setChild(dataset_obj)
# Better to use an 'unloaded' object (loaded = False)
link.setChild(omero.model.DatasetI(dataset_obj.id.val, False))
link.setParent(omero.model.ProjectI(project_Id, False))
conn.getUpdateService().saveObject(link)

- Annotate Image with a new Tag

In [ ]:
tag_ann = omero.gateway.TagAnnotationWrapper(conn)
tag_ann.setValue("Test_Tag")
tag_ann.setDescription("Add optional description")
tag_ann.save()
project = conn.getObject("Image", image_Id)
project.linkAnnotation(tag_ann)

- Add a Map Annotation (list of key: value pairs)

In [ ]:
key_value_data = [["key_1", "value_1"], ["key_2", "value_2"]]
map_ann = omero.gateway.MapAnnotationWrapper(conn)
# Use 'client' namespace to allow editing in Insight & web
namespace = omero.constants.metadata.NSCLIENTMAPANNOTATION
map_ann.setNs(namespace)
map_ann.setValue(key_value_data)
map_ann.save()
project = conn.getObject("Image", image_Id)
# NB: only link a client map annotation to a single object
project.linkAnnotation(map_ann)

- How to create a file annotation and link to ...


In [ ]:
# CSV file
#
image = conn.getObject("Image", image_Id)
# Specify a local file e.g. could be result of some analysis
file_to_upload = "analysis.csv"   # This file should already exist
# create file annotation (uploads the file etc.)
namespace = "my.namespace"
print("\nCreating an FileAnnotation")
file_ann = conn.createFileAnnfromLocalFile(
    file_to_upload, mimetype="text/csv", ns=namespace, desc=None)
print("Attaching FileAnnotation to Dataset: ", "File ID:", file_ann.getId(), \
    ",", file_ann.getFile().getName(), "Size:", file_ann.getFile().getSize())
image.linkAnnotation(file_ann)     # link it to dataset.

In [ ]:
# image
#
image = conn.getObject("Image", image_Id)
# Specify a local file e.g. could be result of some analysis
file_to_upload = "foo.png"   # This file should already exist
# create file annotation (uploads the file etc.)
namespace = "my.namespace"
print("\nCreating an FileAnnotation")
file_ann = conn.createFileAnnfromLocalFile(
    file_to_upload, mimetype="image/png", ns=namespace, desc=None)
print("Attaching FileAnnotation to Dataset: ", "File ID:", file_ann.getId(), \
    ",", file_ann.getFile().getName(), "Size:", file_ann.getFile().getSize())
image.linkAnnotation(file_ann)     # link it to dataset.

- Download a file annotation linked to a Dataset

In [ ]:
# Go through all the annotations on the Dataset. Download any file annotations
# we find. Filter by namespace is optional
print("\nAnnotations on Dataset:", image.getName())
namespace = "my.namespace"
for ann in image.listAnnotations(ns=namespace):
    if isinstance(ann, omero.gateway.FileAnnotationWrapper):
        print("File ID:", ann.getFile().getId(), ann.getFile().getName(), \
            "Size:", ann.getFile().getSize())
        file_path = os.path.join(download_data, ann.getFile().getName())

        with open(str(file_path), 'wb') as f:
            print("\nDownloading file to", file_path, "...")
            for chunk in ann.getFileInChunks():
                f.write(chunk)
        print("File downloaded!")

#### OMERO tables

- set up the table --> 3 columns

In [ ]:
from random import random, sample
table_name = "TablesDemo:%s" % str(random())
col1 = omero.grid.LongColumn('col1', '', [])
col2 = omero.grid.LongColumn('col2', '', [])
col3 = omero.grid.LongColumn('col3', '', [])
columns = [col1, col2, col3]

In [ ]:
resources = conn.c.sf.sharedResources()
repository_id = resources.repositories().descriptions[0].getId().getValue()
table = resources.newTable(repository_id, table_name)
table.initialize(columns)

- produce some random data --> 1000 entries

In [ ]:
ids = list(range(0, 1000, 1))
dat1 = sample(range(1, 5000), 1000)
dat2 = sample(range(1, 5000), 1000)

- fill table

In [ ]:
data1 = omero.grid.LongColumn('col1', '', ids)
data2 = omero.grid.LongColumn('col2', '', dat1)
data3 = omero.grid.LongColumn('col3', '', dat2)
data = [data1, data2, data3]
table.addData(data)
orig_file = table.getOriginalFile()
table.close()           # when we are done, close

- link table to data set

In [ ]:
dataset_Id = 8
orig_file_id = orig_file.id.val
# ...so you can attach this data to an object e.g. Dataset
file_ann = omero.model.FileAnnotationI()
# use unloaded OriginalFileI
file_ann.setFile(omero.model.OriginalFileI(orig_file_id, False))
file_ann = conn.getUpdateService().saveAndReturnObject(file_ann)
link = omero.model.DatasetAnnotationLinkI()
link.setParent(omero.model.DatasetI(dataset_Id, False))
link.setChild(omero.model.FileAnnotationI(file_ann.getId().getValue(), False))
conn.getUpdateService().saveAndReturnObject(link)

#### close connection

In [ ]:
conn.close()